# Project 4: Wrangle and Analyze Data
In this project, i will need to gather dtata from variety of sources and in a variety of formats, assess its quality and tidiness, then clean it. 

**Topic** 
[WeRateDogs](https://en.wikipedia.org/wiki/WeRateDogs) is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10.
The numerators almost always greater than 10. I thas over 4 million followers and has received international media coverage.
The goal of this project is to create interesting and trustworthy analyses and visualizatons based on the data gather from WeRateDogs.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import requests
import os

## For section to use Tweepy library to query the Twitter API for each tweet's JSON
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# 1 Data Wrangling
This section includes the following steps:
- Gathering data 
- Assessing data (detect and document at least 8 quality issues and 2 tidiness issues)
- Cleaning data

## 1.1 Gathering Data
Data collected in this project includes:
1. WeRateDogs twitter archive: twitter_archieve_enhanced.csv
2. Tweet image predictions - breed of dog: image_predictions.tsv
3. Each tweet's retweet count and favourite ("like") count at min

### 1.1.1 WeRateDogs

In [2]:
# Read csv file into DataFrame
twitter_archive = pd.read_csv('./data/twitter-archive-enhanced.csv')
twitter_archive.head()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   
2  <a href="http://twitter.com/download/iphone" r...   
3  <a href="http://twitter.com/download/iphone" r...   
4  <a href="http://twitter.com/download/iphone" r...   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   
2  This is Archie. He is a rare Norwegian Pouncin...                  NaN   
3  This is Darla. She commenced a snooze mid meal...                  NaN   
4  This is Franklin. He would like you to stop ca...                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   
3                       NaN                        NaN   
4                       NaN                        NaN   

                                       expanded_urls  rating_numerator  \
0  https://twitter.com/dog_rates/status/892420643...                13   
1  https://twitter.com/dog_rates/status/892177421...                13   
2  https://twitter.com/dog_rates/status/891815181...                12   
3  https://twitter.com/dog_rates/status/891689557...                13   
4  https://twitter.com/dog_rates/status/891327558...                12   

   rating_denominator      name doggo floofer pupper puppo  
0                  10   Phineas  None    None   None  None  
1                  10     Tilly  None    None   None  None  
2                  10    Archie  None    None   None  None  
3                  10     Darla  None    None   None  None  
4                  10  Franklin  None    None   None  None

### 1.1.2 Image of dog
The file `image_predictions.tsv`, record the prediction the breed of dog that is present in each tweet. It was hosted on Udacity's servers with the url shown in below code cell, which will be downloaded programmatically usign the `Requests` library.

In [5]:
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

response = requests.get(url)

In [7]:
# Open file and write contents of the response vairable to it

with open('./data/image_predictions.tsv', mode = 'wb') as file:
    file.write(response.content)

In [8]:
# Check if it execute successfully
os.listdir('./data')

['twitter-archive-enhanced.csv', '.DS_Store', 'image_predictions.tsv']

In [3]:
image = pd.read_csv('./data/image_predictions.tsv', sep='\t')
image.head()

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3  666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4  666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   
2        1         German_shepherd  0.596461    True            malinois   
3        1     Rhodesian_ridgeback  0.408143    True             redbone   
4        1      miniature_pinscher  0.560311    True          Rottweiler   

    p2_conf  p2_dog                   p3   p3_conf  p3_dog  
0  0.156665    True    Shetland_sheepdog  0.061428    True  
1  0.074192    True  Rhodesian_ridgeback  0.072010    True  
2  0.138584    True           bloodhound  0.116197    True  
3  0.360687    True   miniature_pinscher  0.222752    True  
4  0.243682    True             Doberman  0.154629    True

#### 1.1.3 Additional information
Additional information like tweet's retweet count and favorite ("like") count and any additional interesting data will be gather by querying the **Twitter API** with the tweet IDs from `twitter_archive` dataframe, using Python's [Tweepy](https://www.tweepy.org/) library and store each tweet's entire set of JSON data in `tweet_json.txt`. 
Twitter API

In [3]:
# Install tweepy
#pip install tweepy

  Using cached https://files.pythonhosted.org/packages/bb/7c/99d51f80f3b77b107ebae2634108717362c059a41384a1810d13e2429a81/tweepy-3.9.0-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [4]:
# API Keys, Secrets and Tokens
consumer_key = "YOUR CONSUMER KEY"
consumer_secret = "YOUR CONSUMER SECRET"
access_token = "YOUR ACCESS TOKEN"
access_secret = "YOUR ACCESS SECRET"

The following code, which is provided in the [Getting started](https://buildmedia.readthedocs.org/media/pdf/tweepy/latest/tweepy.pdf) portion of the Tweepy documentation, will create an API object that you can use to gather Twitter data.

In [8]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

The `tweepy` library uses `tweepy.get_status()` to access specific tweets by their id.
```
tweet = api.get_status(id_of_tweet)
print(tweet.text)
```
Describe in [StackOverflow answer](https://stackoverflow.com/questions/28384588/twitter-api-get-tweets-with-specific-id)   
[Reading and Writing JSON to a file in python](https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/)

In [9]:
# Store the tweet_id in twetter_archive dataframe in tweet_ids
tweet_ids = twitter_archive.tweet_id.values
len(tweet_ids)

2356

### DO NOT run below cell code, as this notebook does not include the API keys, secret, token needed

In [10]:
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()

# Save each tweet's returned JSON as a new line in a .txt file
with open('./data/tweet_json_sample.txt', 'w') as outfile:
    # This loop will likely take 20-30 min to run 
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode = 'extended',
                                  wait_on_rate_limit = True, 
                                  wait_on_rate_limit_notify = True)
            
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
        
        
end = timer()
print('Entire query twitter data take {} min'.format(
        (end - start)/60))
print(fails_dict)

1: 892420643555336193
Success
2: 892177421306343426
Success
3: 891815181378084864
Success
4: 891689557279858688
Success
5: 891327558926688256
Success
6: 891087950875897856
Success
7: 890971913173991426
Success
8: 890729181411237888
Success
9: 890609185150312448
Success
10: 890240255349198849
Success
11: 890006608113172480
Success
12: 889880896479866881
Success
13: 889665388333682689
Success
14: 889638837579907072
Success
15: 889531135344209921
Success
16: 889278841981685760
Success
17: 888917238123831296
Success
18: 888804989199671297
Success
19: 888554962724278272
Success
20: 888202515573088257
Fail
21: 888078434458587136
Success
22: 887705289381826560
Success
23: 887517139158093824
Success
24: 887473957103951883
Success
25: 887343217045368832
Success
26: 887101392804085760
Success
27: 886983233522544640
Success
28: 886736880519319552
Success
29: 886680336477933568
Success
30: 886366144734445568
Success
31: 886267009285017600
Success
32: 886258384151887873
Success
33: 8860541600590725

Fail
262: 842846295480000512
Success
263: 842765311967449089
Success
264: 842535590457499648
Success
265: 842163532590374912
Success
266: 842115215311396866
Success
267: 841833993020538882
Success
268: 841680585030541313
Success
269: 841439858740625411
Success
270: 841320156043304961
Success
271: 841314665196081154
Success
272: 841077006473256960
Success
273: 840761248237133825
Success
274: 840728873075638272
Success
275: 840698636975636481
Success
276: 840696689258311684
Success
277: 840632337062862849
Success
278: 840370681858686976
Success
279: 840268004936019968
Success
280: 839990271299457024
Success
281: 839549326359670784
Success
282: 839290600511926273
Success
283: 839239871831150596
Success
284: 838952994649550848
Success
285: 838921590096166913
Success
286: 838916489579200512
Success
287: 838831947270979586
Success
288: 838561493054533637
Success
289: 838476387338051585
Success
290: 838201503651401729
Success
291: 838150277551247360
Success
292: 838085839343206401
Success
293

Success
519: 810657578271330305
Success
520: 810284430598270976
Success
521: 810254108431155201
Success
522: 809920764300447744
Success
523: 809808892968534016
Success
524: 809448704142938112
Success
525: 809220051211603969
Success
526: 809084759137812480
Success
527: 808838249661788160
Success
528: 808733504066486276
Success
529: 808501579447930884
Success
530: 808344865868283904
Success
531: 808134635716833280
Success
532: 808106460588765185
Success
533: 808001312164028416
Success
534: 807621403335917568
Success
535: 807106840509214720
Success
536: 807059379405148160
Success
537: 807010152071229440
Success
538: 806629075125202948
Success
539: 806620845233815552
Success
540: 806576416489959424
Success
541: 806542213899489280
Success
542: 806242860592926720
Success
543: 806219024703037440
Success
544: 805958939288408065
Success
545: 805932879469572096
Success
546: 805826884734976000
Success
547: 805823200554876929
Success
548: 805520635690676224
Success
549: 805487436403003392
Success


Success
776: 776201521193218049
Success
777: 776113305656188928
Success
778: 776088319444877312
Success
779: 775898661951791106
Success
780: 775842724423557120
Success
781: 775733305207554048
Success
782: 775729183532220416
Success
783: 775364825476165632
Success
784: 775350846108426240
Success
785: 775096608509886464
Fail
786: 775085132600442880
Success
787: 774757898236878852
Success
788: 774639387460112384
Success
789: 774314403806253056
Success
790: 773985732834758656
Success
791: 773922284943896577
Success
792: 773704687002451968
Success
793: 773670353721753600
Success
794: 773547596996571136
Success
795: 773336787167145985
Success
796: 773308824254029826
Success
797: 773247561583001600
Success
798: 773191612633579521
Success
799: 772877495989305348
Success
800: 772826264096874500
Success
801: 772615324260794368
Success
802: 772581559778025472
Success
803: 772193107915964416
Success
804: 772152991789019136
Success
805: 772117678702071809
Success
806: 772114945936949249
Success
807

Success
1032: 745422732645535745
Success
1033: 745314880350101504
Success
1034: 745074613265149952
Success
1035: 745057283344719872
Success
1036: 744995568523612160
Success
1037: 744971049620602880
Success
1038: 744709971296780288
Success
1039: 744334592493166593
Success
1040: 744234799360020481
Success
1041: 744223424764059648
Success
1042: 743980027717509120
Success
1043: 743895849529389061
Success
1044: 743835915802583040
Success
1045: 743609206067040256
Success
1046: 743595368194129920
Success
1047: 743545585370791937
Success
1048: 743510151680958465
Success
1049: 743253157753532416
Success
1050: 743222593470234624
Success
1051: 743210557239623680
Success
1052: 742534281772302336
Success
1053: 742528092657332225
Success
1054: 742465774154047488
Success
1055: 742423170473463808
Success
1056: 742385895052087300
Success
1057: 742161199639494656
Success
1058: 742150209887731712
Success
1059: 741793263812808706
Success
1060: 741743634094141440
Success
1061: 741438259667034112
Success
10

...

Success
1530: 690248561355657216
Success
1531: 690021994562220032
Success
1532: 690015576308211712
Success
1533: 690005060500217858
Success
1534: 689999384604450816
Success
1535: 689993469801164801
Success
1536: 689977555533848577
Success
1537: 689905486972461056
Success
1538: 689877686181715968
Success
1539: 689835978131935233
Success
1540: 689661964914655233
Success
1541: 689659372465688576
Success
1542: 689623661272240129
Success
1543: 689599056876867584
Success
1544: 689557536375177216
Success
1545: 689517482558820352
Success
1546: 689289219123089408
Success
1547: 689283819090870273
Success
1548: 689280876073582592
Success
1549: 689275259254616065
Success
1550: 689255633275777024
Success
1551: 689154315265683456
Success
1552: 689143371370250240
Success
1553: 688916208532455424
Success
1554: 688908934925697024
Success
1555: 688898160958271489
Success
1556: 688894073864884227
Success
1557: 688828561667567616
Success
1558: 688804835492233216
Success
1559: 688789766343622656
Success
15

Success
1779: 677895101218201600
Success
1780: 677716515794329600
Success
1781: 677700003327029250
Success
1782: 677698403548192770
Success
1783: 677687604918272002
Success
1784: 677673981332312066
Success
1785: 677662372920729601
Success
1786: 677644091929329666
Success
1787: 677573743309385728
Success
1788: 677565715327688705
Success
1789: 677557565589463040
Success
1790: 677547928504967168
Success
1791: 677530072887205888
Success
1792: 677335745548390400
Success
1793: 677334615166730240
Success
1794: 677331501395156992
Success
1795: 677328882937298944
Success
1796: 677314812125323265
Success
1797: 677301033169788928
Success
1798: 677269281705472000
Success
1799: 677228873407442944
Success
1800: 677187300187611136
Success
1801: 676975532580409345
Success
1802: 676957860086095872
Success
1803: 676949632774234114
Success
1804: 676948236477857792
Success
1805: 676946864479084545
Success
1806: 676942428000112642
Success
1807: 676936541936185344
Success
1808: 676916996760600576
Success
18

Success
2028: 671874878652489728
Success
2029: 671866342182637568
Success
2030: 671855973984772097
Success
2031: 671789708968640512
Success
2032: 671768281401958400
Success
2033: 671763349865160704
Success
2034: 671744970634719232
Success
2035: 671743150407421952
Success
2036: 671735591348891648
Success
2037: 671729906628341761
Success
2038: 671561002136281088
Success
2039: 671550332464455680
Success
2040: 671547767500775424
Success
2041: 671544874165002241
Success
2042: 671542985629241344
Success
2043: 671538301157904385
Success
2044: 671536543010570240
Success
2045: 671533943490011136
Success
2046: 671528761649688577
Success
2047: 671520732782923777
Success
2048: 671518598289059840
Success
2049: 671511350426865664
Success
2050: 671504605491109889
Success
2051: 671497587707535361
Success
2052: 671488513339211776
Success
2053: 671486386088865792
Success
2054: 671485057807351808
Success
2055: 671390180817915904
Success
2056: 671362598324076544
Success
2057: 671357843010908160
Success
20

Rate limit reached. Sleeping for: 40


Success
2144: 669970042633789440
Success
2145: 669942763794931712
Success
2146: 669926384437997569
Success
2147: 669923323644657664
Success
2148: 669753178989142016
Success
2149: 669749430875258880
Success
2150: 669684865554620416
Success
2151: 669683899023405056
Success
2152: 669682095984410625
Success
2153: 669680153564442624
Success
2154: 669661792646373376
Success
2155: 669625907762618368
Success
2156: 669603084620980224
Success
2157: 669597912108789760
Success
2158: 669583744538451968
Success
2159: 669573570759163904
Success
2160: 669571471778410496
Success
2161: 669567591774625800
Success
2162: 669564461267722241
Success
2163: 669393256313184256
Success
2164: 669375718304980992
Success
2165: 669371483794317312
Success
2166: 669367896104181761
Success
2167: 669363888236994561
Success
2168: 669359674819481600
Success
2169: 669354382627049472
Success
2170: 669353438988365824
Success
2171: 669351434509529089
Success
2172: 669328503091937280
Success
2173: 669327207240699904
Success
21

Explore the content in `tweet_json.txt`

In [11]:
# Check the contents of the tweet_json.txt by reading the first tweet
with open('./data/tweet_json.txt', 'r') as json_file:
    first_line = json_file.readline()
    first_data = json.loads(first_line)

# list all the keys
first_data.keys()

dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'extended_entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'possibly_sensitive_appealable', 'lang'])

In [12]:
# check what is info available under 'user''s key
first_data['user']

{'id': 4196983835,
 'id_str': '4196983835',
 'name': 'WeRateDogs®',
 'screen_name': 'dog_rates',
 'location': '',
 'description': 'Your Only Source For Professional Dog Ratings Instagram and Facebook ➪ WeRateDogs partnerships@weratedogs.com ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀',
 'url': 'https://t.co/Ce5NIfoMmj',
 'entities': {'url': {'urls': [{'url': 'https://t.co/Ce5NIfoMmj',
     'expanded_url': 'http://www.amazon.com/thepack',
     'display_url': 'amazon.com/thepack',
     'indices': [0, 23]}]},
  'description': {'urls': []}},
 'protected': False,
 'followers_count': 8883784,
 'friends_count': 19,
 'listed_count': 6001,
 'created_at': 'Sun Nov 15 21:41:29 +0000 2015',
 'favourites_count': 145918,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'verified': True,
 'statuses_count': 13047,
 'lang': None,
 'contributors_enabled': False,
 'is_translator': False,
 'is_translation_enabled': False,
 'profile_background_color': '000000',
 'profile_background_image_url': 'http://abs.twimg.com/images

Read the `tweet_json.txt` file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.

In [4]:
# Collect all selected attributes
selected_attr = []

with open('./data/tweet_json.txt', 'r') as json_file:
    for line in json_file:
        json_data = json.loads(line)
        
        selected_attr.append({
            'tweet_id': json_data['id_str'],
            'favorites': json_data['favorite_count'],
            'user_id': json_data['user']['id_str'],
            'followers_count': json_data['user']['followers_count'],
            'friends_count': json_data['user']['friends_count'],
            'favourites_count': json_data['user']['favourites_count'],
            'statuses_count': json_data['user']['statuses_count'],
            'join_date': json_data['user']['created_at']
        })

In [5]:
# Save all selected attributes in another dataframe
twitter_add = pd.DataFrame(selected_attr, columns = [
    'tweet_id', 'favorites','user_id', 'followers_count',
    'friends_count', 'join_date', 'favourites_count', 'statuses_count'
])

twitter_add.head()

tweet_id  favorites     user_id  followers_count  friends_count  \
0  892420643555336193      35445  4196983835          8883784             19   
1  892177421306343426      30658  4196983835          8883784             19   
2  891815181378084864      23061  4196983835          8883784             19   
3  891689557279858688      38712  4196983835          8883784             19   
4  891327558926688256      36961  4196983835          8883784             19   

                        join_date  favourites_count  statuses_count  
0  Sun Nov 15 21:41:29 +0000 2015            145918           13047  
1  Sun Nov 15 21:41:29 +0000 2015            145918           13047  
2  Sun Nov 15 21:41:29 +0000 2015            145918           13047  
3  Sun Nov 15 21:41:29 +0000 2015            145918           13047  
4  Sun Nov 15 21:41:29 +0000 2015            145918           13047

## 1.2 Access
After obtaining all the data, this section is to assess them to detect any quality and tidiness issues.   
At least 8 quaity issues and 2 tidiness issues are detected. 

### 1.2.1 WeRateDogs

In [6]:
print(f'Shape of twitter_archive : {twitter_archive.shape}')
twitter_archive.head(2)

Shape of twitter_archive : (2356, 17)


tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   

                                       expanded_urls  rating_numerator  \
0  https://twitter.com/dog_rates/status/892420643...                13   
1  https://twitter.com/dog_rates/status/892177421...                13   

   rating_denominator     name doggo floofer pupper puppo  
0                  10  Phineas  None    None   None  None  
1                  10    Tilly  None    None   None  None

#### Data dictionary   
|**No**| **Colume's name** |**Descriptions**|
|:----:|:------------------|:---------------|
|01|created_at|UTC time when this Tweet was created|
|02|id|Unique identifier for this tweet|
|03|source|Utility used to post the tweet, as an html-formatted string|
|04|in_reply_to_status_id|Nullable. If the represented Tweet is a reply, this field will contain the integer representation of the original Tweet's ID|
|05|in_reply_to_user_id|Nullable. If the represented Tweet is a reply, this field will contain the integer representation of the original Tweet's author ID|
|06|Coordinates|Nullable. Represents the geographic location of this Tweet as reported by the user or client application. The inner coordinates array is formatted as `geoJSON`(longitude first, then latitude)|
|07|retweeted_status|Users can amplify the broadcast of Tweets authored by other users by retweeting. Retweets can be distinguished from typical Tweets by the existence of a contains a representation of the original Tweet that was retweeted. Note that retweets of retweets do not show representations of the intermediary retweet, but only the original Tweet|
|08|expanded_url|An expanded version of display_url. Links ot the media display page|
|09|reply_count|Number of times this Tweet has been replied|
|10|retweet_count|Number of times this Tweet has been retweeted|
|11|favorite_count|Nullable. Indicates approximately how many times this Tweet has been **liked** by Tweitter users. This is name as `favorites` in the dataframe|
|12|entities|Entities which have been parsed out the text of the Tweet|
|13|favourites_count|Number of Tweets this user has liked in the account's lifetime. This is under the `user` key|
|14|statuses_count|Number of Tweets (including retweets) issued by the user|
|15|duggo|A big pupper, usually older. This label does not stop a doggo from behaving lke a pupper|
|16|pupper|A small doggo, usually younger. Can be equally, if not more mature than some doggos|
|17|puppo|A transitional phase between pupper and doggo. Easily understood as the dog equivalent to a teenager|
|18|floof|Any dog really. However, this label is commonlly given to dogs with seemingly excess fur. Comical amounts of fur on a dog will certainly earn the dog this generic name|

[Link](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/overview/entities-object#:~:text=Entities%20provide%20metadata%20and%20additional,Twitter%20polls%2C%20and%20attached%20media.)

In [7]:
# List of datatype and null value of all columns
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [8]:
# Statistical description of numeric column
twitter_archive[['rating_numerator','rating_denominator']].describe()

rating_numerator  rating_denominator
count       2356.000000         2356.000000
mean          13.126486           10.455433
std           45.876648            6.745237
min            0.000000            0.000000
25%           10.000000           10.000000
50%           11.000000           10.000000
75%           12.000000           10.000000
max         1776.000000          170.000000

In [9]:
# There is one with '0' as denominator, 
# to impute with most frequent denominator value ('10') in cleaning section
twitter_archive.query('rating_denominator == 0')

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
313  835246439529840640           8.352460e+17           26259576.0   

                     timestamp  \
313  2017-02-24 21:54:03 +0000   

                                                source  \
313  <a href="http://twitter.com/download/iphone" r...   

                                                  text  retweeted_status_id  \
313  @jonnysun @Lin_Manuel ok jomny I know you're e...                  NaN   

     retweeted_status_user_id retweeted_status_timestamp expanded_urls  \
313                       NaN                        NaN           NaN   

     rating_numerator  rating_denominator  name doggo floofer pupper puppo  
313               960                   0  None  None    None   None  None

In [10]:
# There are two row with 0 rating (numerator)
# To refer back to the tweet text to check the rating
twitter_archive.query('rating_numerator == 0')

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
315   835152434251116546                    NaN                  NaN   
1016  746906459439529985           7.468859e+17         4.196984e+09   

                      timestamp  \
315   2017-02-24 15:40:31 +0000   
1016  2016-06-26 03:22:31 +0000   

                                                 source  \
315   <a href="http://twitter.com/download/iphone" r...   
1016  <a href="http://twitter.com/download/iphone" r...   

                                                   text  retweeted_status_id  \
315   When you're so blinded by your systematic plag...                  NaN   
1016  PUPDATE: can't see any. Even if I could, I cou...                  NaN   

      retweeted_status_user_id retweeted_status_timestamp  \
315                        NaN                        NaN   
1016                       NaN                        NaN   

                                          expanded_urls  rating_numerator  \
315   https://twitter.com/dog_rates/status/835152434...                 0   
1016  https://twitter.com/dog_rates/status/746906459...                 0   

      rating_denominator  name doggo floofer pupper puppo  
315                   10  None  None    None   None  None  
1016                  10  None  None    None   None  None

Based on the text from the tweet text, the 2 are truely with 0 rating

In [11]:
# Tweet text on first tweet wiht 0 rating
twitter_archive.loc[315, 'text']

"When you're so blinded by your systematic plagiarism that you forget what day it is. 0/10 https://t.co/YbEJPkg4Ag"

In [12]:
# Tweet text on first tweet wiht 0 rating
twitter_archive.loc[1016, 'text']

"PUPDATE: can't see any. Even if I could, I couldn't reach them to pet. 0/10 much disappointment https://t.co/c7WXaB2nqX"

### 1.2.2 Image of dog
Out of 2356 rows in twitter_archive, there is only 2075 rows with image of dog dataframe.

In [13]:
print(f'Shape of image : {image.shape}')
image.head(2)

Shape of image : (2075, 12)


tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   

    p2_conf  p2_dog                   p3   p3_conf  p3_dog  
0  0.156665    True    Shetland_sheepdog  0.061428    True  
1  0.074192    True  Rhodesian_ridgeback  0.072010    True

In [13]:
image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [15]:
# Majority of the dog prediction with high confidence level is from p1
# Majority of tweet with 1 image
image.describe()

tweet_id      img_num      p1_conf       p2_conf       p3_conf
count  2.075000e+03  2075.000000  2075.000000  2.075000e+03  2.075000e+03
mean   7.384514e+17     1.203855     0.594548  1.345886e-01  6.032417e-02
std    6.785203e+16     0.561875     0.271174  1.006657e-01  5.090593e-02
min    6.660209e+17     1.000000     0.044333  1.011300e-08  1.740170e-10
25%    6.764835e+17     1.000000     0.364412  5.388625e-02  1.622240e-02
50%    7.119988e+17     1.000000     0.588230  1.181810e-01  4.944380e-02
75%    7.932034e+17     1.000000     0.843855  1.955655e-01  9.180755e-02
max    8.924206e+17     4.000000     1.000000  4.880140e-01  2.734190e-01

In [16]:
# There is non-dog breed prediction in p1 prediction, 
# thus, need to perform cleaning to extract all dog breed prediction
image.p1_dog.unique()

array([ True, False])

In [17]:
# Inconsistency in the dog breed, some with capitalize, some with all small capital
image.query('p1_dog == True').p1.unique()

array(['Welsh_springer_spaniel', 'redbone', 'German_shepherd',
       'Rhodesian_ridgeback', 'miniature_pinscher',
       'Bernese_mountain_dog', 'chow', 'miniature_poodle',
       'golden_retriever', 'Gordon_setter', 'Walker_hound', 'pug',
       'bloodhound', 'Lhasa', 'English_setter', 'Italian_greyhound',
       'Maltese_dog', 'malamute', 'soft-coated_wheaten_terrier',
       'Chihuahua', 'black-and-tan_coonhound', 'toy_terrier',
       'Blenheim_spaniel', 'Pembroke', 'Chesapeake_Bay_retriever',
       'curly-coated_retriever', 'dalmatian', 'Ibizan_hound',
       'Border_collie', 'Labrador_retriever', 'miniature_schnauzer',
       'Airedale', 'West_Highland_white_terrier', 'toy_poodle',
       'giant_schnauzer', 'vizsla', 'Rottweiler', 'Siberian_husky',
       'papillon', 'Saint_Bernard', 'Tibetan_terrier', 'borzoi', 'beagle',
       'Yorkshire_terrier', 'Pomeranian', 'kuvasz',
       'flat-coated_retriever', 'Norwegian_elkhound', 'standard_poodle',
       'Staffordshire_bullterrier

### 1.2.3 twitter_add
This contains additional information like retweet count and favorite count.
Out of 2356 rows in twitter_archive,there is only 2327 rows in twitter_add dataframe.

In [18]:
print(f'Shape of twitter_add : {twitter_add.shape}')
twitter_add.head(2)

Shape of twitter_add : (2329, 8)


tweet_id  favorites     user_id  followers_count  friends_count  \
0  892420643555336193      35445  4196983835          8883784             19   
1  892177421306343426      30658  4196983835          8883784             19   

                        join_date  favourites_count  statuses_count  
0  Sun Nov 15 21:41:29 +0000 2015            145918           13047  
1  Sun Nov 15 21:41:29 +0000 2015            145918           13047

In [19]:
twitter_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2329 entries, 0 to 2328
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tweet_id          2329 non-null   object
 1   favorites         2329 non-null   int64 
 2   user_id           2329 non-null   object
 3   followers_count   2329 non-null   int64 
 4   friends_count     2329 non-null   int64 
 5   join_date         2329 non-null   object
 6   favourites_count  2329 non-null   int64 
 7   statuses_count    2329 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 145.7+ KB


### Quality
`twitter_archive` **table**
1. timestamp and retweeted_status_timestamp are in str format, not datetime format
2. source is in html formtted string, instead of the name of the utility used to post the tweet
3. rating denomerator with min value of 0
4. Remove all retweet.
5. Remove all original ratings that do not have images   
6. Some rows without dog stage (null value), change it to none 
7. all id shall be in str format instead of numeric format, as it doesn't have numeric meaning

`image` **table** 
8. Remove the row with non-breed prediction (that is all false for p1, p2, & p3). In addition, record only ONE breed/type, based on the highest confidence level prediciton.
9. In-consistence in dog breed structure. Some with capitalized, some with all lower letters. To capitalize all dog breeed.

`twitter_add` **table**
10. join_date is in str format, not datetime format


### Tidiness
`twitter_archive` **table**
1. timestamp, split into two column with one column for date, and another column for time
2. There are 4 columns to state the dog stage (doggo, floofer, pupper, puppo) and also none type. To combine all into single column as dog_stage.
3. The prediction of dog breeds. Thus, need to consolidate a single column with the dog breeds  
`twitter_archieve_master` **clean table**
4. twitter_add and dog breed prediction should be added to the main twitter_archive, and save it into new dataframe, named `twitter_archieve_master`.
5. Drop unwanted column (column with null value), and create a new column for the dog rating

## 1.3 Clean

In [7]:
# create a new copy of dataframe for cleaning
twitter_clean = twitter_archive.copy()
image_clean = image.copy()
twitter_add_clean = twitter_add.copy()

### 1.3.1 Timestamp
`twitter_archive` and `twitter_add` tables   
**Define**   
timestamp is in str format, to convert it to datetime format. (This is quality issue)   
Once in datetime format, I can create additional 2 columns to parse the datetime into date and time in separate column. (This is *tidiness issue*)

**Code**

In [8]:
# convert timesteamp str datatype to datetime datatype
twitter_clean['timestamp'] = pd.to_datetime(twitter_clean['timestamp'])
twitter_add_clean['join_timestamp'] = pd.to_datetime(twitter_add_clean.join_date)

# create new column to for tweet_date & tweet_time
twitter_clean['tweet_date'] = [x.date() for x in twitter_clean['timestamp']]
twitter_clean['tweet_time'] = [x.time() for x in twitter_clean['timestamp']]
twitter_add_clean['join_date'] = [x.date() for x in twitter_add_clean['join_timestamp']]

**Test**

In [9]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   int64              
 1   in_reply_to_status_id       78 non-null     float64            
 2   in_reply_to_user_id         78 non-null     float64            
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         181 non-null    float64            
 7   retweeted_status_user_id    181 non-null    float64            
 8   retweeted_status_timestamp  181 non-null    object             
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

In [10]:
twitter_add_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2329 entries, 0 to 2328
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   tweet_id          2329 non-null   object             
 1   favorites         2329 non-null   int64              
 2   user_id           2329 non-null   object             
 3   followers_count   2329 non-null   int64              
 4   friends_count     2329 non-null   int64              
 5   join_date         2329 non-null   object             
 6   favourites_count  2329 non-null   int64              
 7   statuses_count    2329 non-null   int64              
 8   join_timestamp    2329 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(5), object(3)
memory usage: 163.9+ KB


### 1.3.2 source column
`twitter_archive` table   
**Define**   
`source` is in html formtted string, instead of the name of the utility used to post the tweet. 
```
'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>'
```
To extract the utility, i.e. *Twitter for iPhone* as shown in above example

**Code**

In [11]:
twitter_clean['source'] = twitter_clean.source.str.extract(r'rel="nofollow">(.*)<')


**Test**

In [12]:
twitter_clean.source.unique()

array(['Twitter for iPhone', 'Twitter Web Client', 'Vine - Make a Scene',
       'TweetDeck'], dtype=object)

### 1.3.3 rating_denominator column
`twitter_archive` table   
**Define**   
There is one rating denomerator with the value of 0. To check the text content.
Suspected this could be an error and to find out the correct rating if any from the tweet's content.

**Code**

In [13]:
# Check the text content with rating_denominator = 0
twitter_clean.loc[twitter_clean.query('rating_denominator == 0').index.to_list()]

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
313  835246439529840640           8.352460e+17           26259576.0   

                    timestamp              source  \
313 2017-02-24 21:54:03+00:00  Twitter for iPhone   

                                                  text  retweeted_status_id  \
313  @jonnysun @Lin_Manuel ok jomny I know you're e...                  NaN   

     retweeted_status_user_id retweeted_status_timestamp expanded_urls  \
313                       NaN                        NaN           NaN   

     rating_numerator  rating_denominator  name doggo floofer pupper puppo  \
313               960                   0  None  None    None   None  None   

     tweet_date tweet_time  
313  2017-02-24   21:54:03

The row with rating_denominator = 0 is **313** as shown above.
I check the content of the tweet and found out that the valid rating 
should be **13/10** instead of 960/00

In [14]:
twitter_clean.loc[313,'text']

"@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho"

In [15]:
# change from 960 to 13 for the rating numerator
twitter_clean.loc[313,'rating_numerator'] = 13

# impute 0 with 10
twitter_clean['rating_denominator'] = twitter_clean.rating_denominator.replace({0:10})

**Test**

In [16]:
# no denominator with 0
twitter_clean.query('rating_denominator == 0')

Empty DataFrame
Columns: [tweet_id, in_reply_to_status_id, in_reply_to_user_id, timestamp, source, text, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, expanded_urls, rating_numerator, rating_denominator, name, doggo, floofer, pupper, puppo, tweet_date, tweet_time]
Index: []

### 1.3.4 remove all retweet
`twitter_archive` table   
**Define**   
As I only interested in the original ratings (no retweets) that have images, thus I need to remove all retweets in the dataset.   
For this, 
- remove all rows that `retweeted_status_id` is not null (indicated as retweet)
- remove all rows that `in_reply_to_status_id` is not null (indicated is a reply)

**Code**

In [17]:
# Retrieve only original tweets, i.e. retweeted_status_id is Null
twitter_clean = twitter_clean[twitter_clean.retweeted_status_id.isnull()]
twitter_clean = twitter_clean[twitter_clean.in_reply_to_status_id.isnull()]

**Test**

In [18]:
twitter_clean.retweeted_status_id.unique()

array([nan])

In [19]:
twitter_clean.in_reply_to_status_id.unique()

array([nan])

### 1.3.5 Remove tweet without image
`twitter_archive` table   
**Define**   
Remove all original ratings that do not have images, that is remove all row in column `expanded_urls` that is null

**Code**

In [20]:
twitter_clean = twitter_clean[twitter_clean.expanded_urls.notnull()]
twitter_clean.shape

(2094, 19)

**Test**

In [21]:
# Count the row with null value in columns `expanded_urls`
twitter_clean.expanded_urls.isnull().sum()

0

### 1.3.6 Multiple column with dog stage info (doggo, floofer, pupper, puppo)
`twitter_archive` table   
**Define**   
This is tidyness issue
There are 4 columns to state the dog stage (doggo, floofer, pupper, puppo) and also none type. To combine all into single column as dog_stage.   
*This is tidiness issue*

**Code**

In [22]:
# replace all 'None' value in these 4 columns with empty string ''
# combine all the value in these 4 columns, (by row)
# check the unique value of the results
twitter_clean[['doggo', 'floofer', 'pupper', 'puppo']].replace('None','').sum(1).unique()

array(['', 'doggo', 'puppo', 'pupper', 'floofer', 'doggopuppo',
       'doggofloofer', 'doggopupper'], dtype=object)

There are some rows with 2 entries within these 4 columns:
1. doggopuppo
2. doogofloofer
3. doggopupper   

Using code cell below, it is able to identify the actual dog stage of that row by the tweet text.
Thus, to compress the cell into single column, name as 'dog_stage' and manual clean up those row with multiple dog stage

In [23]:
# check row with 'doggopuppo'. Based on the text, the dog stage should be puppo instead of doggo
twitter_clean.query('doggo == "doggo" and puppo == "puppo"')

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
191  855851453814013952                    NaN                  NaN   

                    timestamp              source  \
191 2017-04-22 18:31:02+00:00  Twitter for iPhone   

                                                  text  retweeted_status_id  \
191  Here's a puppo participating in the #ScienceMa...                  NaN   

     retweeted_status_user_id retweeted_status_timestamp  \
191                       NaN                        NaN   

                                         expanded_urls  rating_numerator  \
191  https://twitter.com/dog_rates/status/855851453...                13   

     rating_denominator  name  doggo floofer pupper  puppo  tweet_date  \
191                  10  None  doggo    None   None  puppo  2017-04-22   

    tweet_time  
191   18:31:02

In [24]:
twitter_clean.loc[191,'text']

"Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel"

In [25]:
#create new column to list the dog stage by row
twitter_clean['dog_stage'] = twitter_clean[['doggo', 'floofer', 'pupper', 'puppo']].replace('None','').sum(1)


# drop unwanted column: 'doggo', 'floofer', 'pupper', 'puppo'
twitter_clean.drop(columns = ['doggo', 'floofer', 'pupper', 'puppo'], inplace = True)


Manual clean up for `doggopuppo`

In [26]:
twitter_clean.loc[191,'dog_stage'] = 'puppo'

Manual clean up for `doogofloofer`

In [27]:
twitter_clean.loc[twitter_clean.query('dog_stage == "doggofloofer"').index.to_list()[0],'text']

"At first I thought this was a shy doggo, but it's actually a Rare Canadian Floofer Owl. Amateurs would confuse the two. 11/10 only send dogs https://t.co/TXdT3tmuYk"

In [28]:
twitter_clean.loc[twitter_clean.query('dog_stage == "doggofloofer"').index.to_list()[0],'dog_stage'] = 'floofer'

Manual clean up for `doggopupper`

In [29]:
# List out all the row number with dog stage of doggo & pupper
twitter_clean.query('dog_stage == "doggopupper"').index.to_list()

[460, 531, 575, 705, 733, 889, 956, 1063, 1113]

In [30]:
# This should be doggo
print(twitter_clean.loc[460, 'text'])

# change to doggo
twitter_clean.loc[460,'dog_stage'] = 'doggo'

This is Dido. She's playing the lead role in "Pupper Stops to Catch Snow Before Resuming Shadow Box with Dried Apple." 13/10 (IG: didodoggo) https://t.co/m7isZrOBX7


In [31]:
# No change, as there are two dogs, one for pupper, one for doggo
twitter_clean.loc[531, 'text']

'Here we have Burke (pupper) and Dexter (doggo). Pupper wants to be exactly like doggo. Both 12/10 would pet at same time https://t.co/ANBpEYHaho'

In [32]:
# This should be pupper
print(twitter_clean.loc[575, 'text'])

# change to pupper
twitter_clean.loc[575, 'dog_stage'] = 'pupper'

This is Bones. He's being haunted by another doggo of roughly the same size. 12/10 deep breaths pupper everything's fine https://t.co/55Dqe0SJNj


In [33]:
# This should be doggo
print(twitter_clean.loc[705, 'text'])

# change to doggo
twitter_clean.loc[705,'dog_stage'] = 'doggo'

This is Pinot. He's a sophisticated doggo. You can tell by the hat. Also pointier than your average pupper. Still 10/10 would pet cautiously https://t.co/f2wmLZTPHd


In [34]:
# No change, consist of doggo and pupper
twitter_clean.loc[733, 'text']

'Pupper butt 1, Doggo 0. Both 12/10 https://t.co/WQvcPEpH2u'

In [35]:
# No change, consist of doggo and pupper
twitter_clean.loc[889, 'text']

'Meet Maggie &amp; Lila. Maggie is the doggo, Lila is the pupper. They are sisters. Both 12/10 would pet at the same time https://t.co/MYwR4DQKll'

In [36]:
# Refer to the tweet reply via the extended_urls, this should be doggo

print(twitter_clean.loc[956, 'text'])

# change to doggon
twitter_clean.loc[956,'dog_stage'] = 'doggo'

Please stop sending it pictures that don't even have a doggo or pupper in them. Churlish af. 5/10 neat couch tho https://t.co/u2c9c7qSg8


In [37]:
# No change, consist of doggo and pupper
twitter_clean.loc[1063, 'text']

'This is just downright precious af. 12/10 for both pupper and doggo https://t.co/o5J479bZUC'

In [38]:
# No change, consist of doggo and pupper
twitter_clean.loc[1113, 'text']

'Like father (doggo), like son (pupper). Both 12/10 https://t.co/pG2inLaOda'

**test**

In [39]:
# There should have 5 doggopupper.
twitter_clean.dog_stage.value_counts()

               1759
pupper          221
doggo            75
puppo            24
floofer          10
doggopupper       5
Name: dog_stage, dtype: int64

### 1.3.7 Some rows without dog stage (null value), change it to None
`twitter_archive` table   
**Define**   
From above code cell, majority is tweet without dog stage, and is save as empty string. Thus, to change it to `None` instead. 

**Code**

In [40]:
twitter_clean.dog_stage.replace('','None', inplace = True)

**test**

In [41]:
twitter_clean.dog_stage.value_counts()

None           1759
pupper          221
doggo            75
puppo            24
floofer          10
doggopupper       5
Name: dog_stage, dtype: int64

### 1.3.8 inappropriate datatype for id column
`twitter_archive` table and `image_prediction` table

**Define**   
All id shall be in str format instead of numeric format, as it doesn't have numeric meaning

**Code**

In [42]:
twitter_clean['tweet_id'] = twitter_clean.tweet_id.astype('str')

image_clean['tweet_id'] = image_clean.tweet_id.astype('str')

**test**

In [43]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2094 entries, 0 to 2355
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2094 non-null   object             
 1   in_reply_to_status_id       0 non-null      float64            
 2   in_reply_to_user_id         0 non-null      float64            
 3   timestamp                   2094 non-null   datetime64[ns, UTC]
 4   source                      2094 non-null   object             
 5   text                        2094 non-null   object             
 6   retweeted_status_id         0 non-null      float64            
 7   retweeted_status_user_id    0 non-null      float64            
 8   retweeted_status_timestamp  0 non-null      object             
 9   expanded_urls               2094 non-null   object             
 10  rating_numerator            2094 non-null   int64           

In [77]:
image_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1751 entries, 0 to 2073
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   1751 non-null   object
 1   dog_breed  1751 non-null   object
dtypes: object(2)
memory usage: 121.0+ KB


### 1.3.9 Clean up the dog breed/ type prediction
`image` table   

**Define**   
Remove the row with non-breed prediction (that is all false for p1, p2, & p3). In addition, record only ONE breed/type, based on the highest confidence level prediciton, and save in column `dog_breed`   
*This is Tidiness issue*.

**Code**

In [44]:
# Mask for p1, p2, p3 non dog prediction
mask_1 = image_clean['p1_dog'] == False
mask_2 = image_clean['p2_dog'] == False
mask_3 = image_clean['p3_dog'] == False

In [45]:
# There are total of 324 rows with non dog prediction
image_clean[mask_1 & mask_2 & mask_3]

tweet_id                                            jpg_url  \
6     666051853826850816    https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg   
17    666104133288665088    https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg   
18    666268910803644416    https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg   
21    666293911632134144    https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg   
25    666362758909284353    https://pbs.twimg.com/media/CT9lXGsUcAAyUFt.jpg   
...                  ...                                                ...   
2021  880935762899988482    https://pbs.twimg.com/media/DDm2Z5aXUAEDS2u.jpg   
2022  881268444196462592    https://pbs.twimg.com/media/DDrk-f9WAAI-WQv.jpg   
2046  886680336477933568    https://pbs.twimg.com/media/DE4fEDzWAAAyHMM.jpg   
2052  887517139158093824  https://pbs.twimg.com/ext_tw_video_thumb/88751...   
2074  892420643555336193    https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg   

      img_num                p1   p1_conf  p1_dog               p2   p2_conf  \
6           1        box_turtle  0.933012   False       mud_turtle  0.045885   
17          1               hen  0.965932   False             cock  0.033919   
18          1  desktop_computer  0.086502   False             desk  0.085547   
21          1  three-toed_sloth  0.914671   False            otter  0.015250   
25          1        guinea_pig  0.996496   False            skunk  0.002402   
...       ...               ...       ...     ...              ...       ...   
2021        1       street_sign  0.251801   False         umbrella  0.115123   
2022        1            tusker  0.473303   False  Indian_elephant  0.245646   
2046        1       convertible  0.738995   False       sports_car  0.139952   
2052        1         limousine  0.130432   False        tow_truck  0.029175   
2074        1            orange  0.097049   False            bagel  0.085851   

      p2_dog              p3   p3_conf  p3_dog  
6      False        terrapin  0.017885   False  
17     False       partridge  0.000052   False  
18     False        bookcase  0.079480   False  
21     False  great_grey_owl  0.013207   False  
25     False         hamster  0.000461   False  
...      ...             ...       ...     ...  
2021   False   traffic_light  0.069534   False  
2022   False            ibex  0.055661   False  
2046   False       car_wheel  0.044173   False  
2052   False   shopping_cart  0.026321   False  
2074   False          banana  0.076110   False  

[324 rows x 12 columns]

In [46]:
# remove all rows with non dog prediction
image_clean = image_clean[~(mask_1 & mask_2 & mask_3)]

In [47]:
# Loop through each row, and extract the dog breed if it is predicted to be dog,
# starting from p1, then p2, then p3, as it started with the highest confidence level prediction
for idx,row in image_clean.iterrows():
    if row['p1_dog'] == True:
        image_clean.at[idx,'dog_breed'] = row['p1']
        
    elif row['p2_dog'] == True:
        image_clean.at[idx,'dog_breed'] = row['p2']
    
    elif row['p3_dog'] == True:
        image_clean.at[idx,'dog_breed'] = row['p3']
        

In [48]:
# Keep only the wanted column
image_clean = image_clean[['tweet_id','dog_breed']]

**Test**

In [53]:
image_clean.head()

tweet_id               dog_breed
0  666020888022790149  Welsh_springer_spaniel
1  666029285002620928                 redbone
2  666033412701032449         German_shepherd
3  666044226329800704     Rhodesian_ridgeback
4  666049248165822465      miniature_pinscher

### 1.3.10 In-consistence dog breed naming structure
`image` table   

**Define**   
In-consistence in dog breed structure. Some with capitalized, some with all lower letters. To capitalize all dog breeed.


**Code**

In [57]:
image_clean['dog_breed'] = [name.capitalize() for name in image_clean.dog_breed]

**Test**

In [58]:
image_clean.head()

tweet_id               dog_breed
0  666020888022790149  Welsh_springer_spaniel
1  666029285002620928                 Redbone
2  666033412701032449         German_shepherd
3  666044226329800704     Rhodesian_ridgeback
4  666049248165822465      Miniature_pinscher

In [103]:
# Export image_clean to csv file
image_clean.to_csv('./output/image_clean.csv', index = False)
twitter_clean.to_csv('./output/twitter_clean.csv', index = False)

### 1.3.11 Combine twitter_add, image_prediction to the main twitter_archieve
`twitter_add` table   

**Define**   
Combine `twitter_add` and `image_prediction` table into main `twitter_archieve` table. 
Save it as clean dataframe name `twitter_archieve_master`.       
*Tidiness issue*

**Code**

In [87]:
# Display number of rows and column for all the 3 tables
print(f'twitter_archieve: {twitter_clean.shape}')
print(f'image_prediction: {image_clean.shape}')
print(f'twitter_add: {twitter_add_clean.shape}')

twitter_archieve: (2094, 16)
image_prediction: (1751, 2)
twitter_add: (2329, 9)


In [92]:
# Merge twitter_archieve with image_prediction
# Left join to have all the rows in the twitter_archieve, as not all has the image_prediction
twitter_archieve_master = twitter_clean.merge(image_clean, how = 'left', on = 'tweet_id')

# Merge the above with twitter_add
# Inner join, to remove the row without additional info from twitter_add table
twitter_archieve_master = twitter_archieve_master.merge(twitter_add_clean, 
                                                        how = 'inner', on = 'tweet_id')

print(f'twitter_archieve_master: {twitter_archieve_master.shape}')

twitter_archieve_master: (2085, 25)


**Test**

In [93]:
twitter_archieve_master.head()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                  timestamp              source  \
0 2017-08-01 16:23:56+00:00  Twitter for iPhone   
1 2017-08-01 00:17:27+00:00  Twitter for iPhone   
2 2017-07-31 00:18:03+00:00  Twitter for iPhone   
3 2017-07-30 15:58:51+00:00  Twitter for iPhone   
4 2017-07-29 16:00:24+00:00  Twitter for iPhone   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   
2  This is Archie. He is a rare Norwegian Pouncin...                  NaN   
3  This is Darla. She commenced a snooze mid meal...                  NaN   
4  This is Franklin. He would like you to stop ca...                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   
3                       NaN                        NaN   
4                       NaN                        NaN   

                                       expanded_urls  ...  dog_stage  \
0  https://twitter.com/dog_rates/status/892420643...  ...       None   
1  https://twitter.com/dog_rates/status/892177421...  ...       None   
2  https://twitter.com/dog_rates/status/891815181...  ...       None   
3  https://twitter.com/dog_rates/status/891689557...  ...       None   
4  https://twitter.com/dog_rates/status/891327558...  ...       None   

            dog_breed favorites     user_id followers_count friends_count  \
0                 NaN     35445  4196983835         8883784            19   
1           Chihuahua     30658  4196983835         8883784            19   
2           Chihuahua     23061  4196983835         8883784            19   
3  Labrador_retriever     38712  4196983835         8883784            19   
4              Basset     36961  4196983835         8883784            19   

    join_date  favourites_count statuses_count            join_timestamp  
0  2015-11-15            145918          13047 2015-11-15 21:41:29+00:00  
1  2015-11-15            145918          13047 2015-11-15 21:41:29+00:00  
2  2015-11-15            145918          13047 2015-11-15 21:41:29+00:00  
3  2015-11-15            145918          13047 2015-11-15 21:41:29+00:00  
4  2015-11-15            145918          13047 2015-11-15 21:41:29+00:00  

[5 rows x 25 columns]

In [94]:
twitter_archieve_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2085 entries, 0 to 2084
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2085 non-null   object             
 1   in_reply_to_status_id       0 non-null      float64            
 2   in_reply_to_user_id         0 non-null      float64            
 3   timestamp                   2085 non-null   datetime64[ns, UTC]
 4   source                      2085 non-null   object             
 5   text                        2085 non-null   object             
 6   retweeted_status_id         0 non-null      float64            
 7   retweeted_status_user_id    0 non-null      float64            
 8   retweeted_status_timestamp  0 non-null      object             
 9   expanded_urls               2085 non-null   object             
 10  rating_numerator            2085 non-null   int64           

### 1.3.12 Drop unwanted column, and create a new column for the dog rating
`twitter_archieve_master` table   

**Define**   
Drop unwanted column, i.e. those column with all NULL value (in_reply_to_status_id,
in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id retweeted_status_timestamp).   
Next is to calculate the dog rating by rating_numerator/ rating_denominator    
*Tidiness issue*

**Code**

In [96]:
# Drop unwanted columns
twitter_archieve_master.drop(columns = ['in_reply_to_status_id', 'in_reply_to_user_id',
        'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'],
                            inplace = True)

In [98]:
# Add a column to calculate the dog rating
twitter_archieve_master['rating'] = twitter_archieve_master['rating_numerator']/twitter_archieve_master['rating_denominator']

In [102]:
# Export clean dataframe to csv file
twitter_archieve_master.to_csv('./output/twitter_archieve_master.csv', index = False)

**Test**

In [99]:
twitter_archieve_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2085 entries, 0 to 2084
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            2085 non-null   object             
 1   timestamp           2085 non-null   datetime64[ns, UTC]
 2   source              2085 non-null   object             
 3   text                2085 non-null   object             
 4   expanded_urls       2085 non-null   object             
 5   rating_numerator    2085 non-null   int64              
 6   rating_denominator  2085 non-null   int64              
 7   name                2085 non-null   object             
 8   tweet_date          2085 non-null   object             
 9   tweet_time          2085 non-null   object             
 10  dog_stage           2085 non-null   object             
 11  dog_breed           1658 non-null   object             
 12  favorites           2085 non-null 

# 2 Exploratory Data Analysis
Analyze and visualize with at least 3 insights and 1 visualization.

**Questions**
1. Which is the common platform for user to push their tweet?
2. How's the rating distribution
3. Correlation between rating with **like** (favorites) counts
4. What dog breed is most popular (averagely having high number of favorites, **like** counts)
5. The dog'stage distribution and which dog stage is most popular (averagely having high number of likes).
6. What is the tweet's text with top 5 rating
7. User distribution in terms of how many tweet per user
8. Any correlation between the number of **like** in the tweet with the number of **like** the user like in their twitter account?
9. Any correlation between the number of **like** in the tweet with the number of **followers** the user have?
10. Any correlation between the number of **like** in the tweet with the number of **friends** the user have?